In [8]:
import logging
import sklearn.datasets as ds
import pandas as pd
import numpy as np
import math as mt

import time
import datetime

from tensorflow.keras.optimizers import Adam
from itertools import product
from sklearn.metrics import mean_squared_error
from mulearn import FuzzyInductor
from mulearn.optimization import TensorFlowSolver
from mulearn.kernel import GaussianKernel

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [9]:
def create_dataset(name):

    #load dataset, in iris_X values, in iris_y labels 0 1 2
    iris_X, iris_y = ds.load_iris(return_X_y=True)  

    labels = ("Setosa", "Versicolor", "Virginica") 

    #dataframe with correct labels for respective values
    df = pd.DataFrame(iris_X, columns=["Sepal length", "Sepal width",
                                       "Petal length", "Petal width"])

    #associating 
    df['Class'] = iris_y
    df['Class'] = df['Class'].map(lambda c: labels[c])

    #dataset copy for labels 0 1
    selected_iris_dataset = iris_y.copy()

    #dataset selected with labels
    if(name == "Setosa"):        
        selected_iris_dataset[selected_iris_dataset != 0] = 2
        selected_iris_dataset[selected_iris_dataset == 0] = 1
        selected_iris_dataset[selected_iris_dataset == 2] = 0
    elif(name == "Versicolor"):
        selected_iris_dataset[selected_iris_dataset==2] = 0    
    elif(name == "Virginica"):
        selected_iris_dataset[selected_iris_dataset != 2] = 0
        selected_iris_dataset[selected_iris_dataset == 2] = 1


    return iris_X, selected_iris_dataset

In [10]:
def create_handler(path):   
    
    
    fhandler = logging.FileHandler(filename = path)
    
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    
    logger.addHandler(fhandler)  
    
    return fhandler

In [11]:
def main(nome, c, sigma, penalization, learning_rate):  
    
    handler = create_handler(f"../../../log/Prove-4/N_Iter_3000/{nome}/c{str(c).replace('.','')}"
                             f"_sigma{str(sigma).replace('.','')}"
                             f"_penalization{str(penalization).replace('.','')}"
                             f"_{str(learning_rate)}.log")
    
    #salvo i parametri
    logger.info(f"PARAMETRI DI PARTENZA: nome={nome}, c={c}, sigma={sigma}, penalization={penalization},"
                f"learning rate={learning_rate}")
    
    

    iris_X, selected_iris_dataset = create_dataset(nome)
    
    n_iter = 100    

    # Gurobi solver & fit
    fi = FuzzyInductor(c=c, k=GaussianKernel(sigma=sigma))
    
    
    start = time.time()
    fi.fit(iris_X, selected_iris_dataset)
    end = time.time()
    
    # rmse gurobi
    gurobi_chis = fi.chis_
    logger.info(f"GUROBI_CHIS: {gurobi_chis}")
    logger.info(f"GUROBI_START: {start}")
    logger.info(f"GUROBI_END: {end}")
    logger.info(f"TEMPO_ESECUZIONE GUROBI(IN EPOCH): {(end-start)}")

    # TensorFlow solver
    fi = FuzzyInductor(solver=TensorFlowSolver(n_iter=n_iter, optimizer=Adam(learning_rate=learning_rate),    
                                               penalization=penalization),c=c,k=GaussianKernel(sigma=sigma))
    
    try:
        start = time.time()
        fi.fit(iris_X, selected_iris_dataset)
        end = time.time()
    except (ModuleNotFoundError, ValueError):
        print('Tensorflow not available')


    # rmse TensorFlow
    tensorflow_chis = fi.chis_
    logger.info(f"TENSORFLOW_CHIS: {tensorflow_chis}") 
    logger.info(f"TENSORFLOW_START: {start}")
    logger.info(f"TENSORFLOW_END: {end}")
    
    total_tensorflow = end-start
    logger.info(f"TEMPO PER ITERAZIONI (N_ITER,TEMPO) TENSORFLOW: [({n_iter},{total_tensorflow})]")

    # calcolo distanza
    rmse_distance = abs(mean_squared_error(gurobi_chis, tensorflow_chis, squared=False))
    logger.info(f"RMSE_DISTANCE_CHIS: {rmse_distance}")

    #coppia n_iter, distance
    couple = [(n_iter, rmse_distance)]
    logger.info(f"COUPLE(N_ITER,DISTANCE RMSE):  {couple}")

    #incremento n
    n = 200

    # faccio ciclo fino a 10000
    while n <= 3000:
        
        chi_ = tensorflow_chis
        
        # TensorFlow solver
        fi = FuzzyInductor(solver=TensorFlowSolver(initial_values=chi_, n_iter=n_iter, 
                                                   optimizer=Adam(learning_rate=learning_rate),
                                                   penalization=penalization),c=c,k=GaussianKernel(sigma=sigma))
        try:
            start = time.time()
            fi.fit(iris_X, selected_iris_dataset)
            end = time.time()
        except (ModuleNotFoundError, ValueError):
            print('Tensorflow not available')

        total_tensorflow += (end-start)
        
        # rmse TensorFlow
        tensorflow_chis = fi.chis_
        logger.info(f"TENSORFLOW_CHIS: {tensorflow_chis}")
        logger.info(f"TENSORFLOW_START: {start}")
        logger.info(f"TENSORFLOW_END: {end}")
        logger.info(f"TEMPO PER ITERAZIONI (N_ITER,TEMPO) TENSORFLOW: [({n},{total_tensorflow})]")
        

        # calcolo distanza
        rmse_distance = abs(mean_squared_error(gurobi_chis, tensorflow_chis, squared=False))
        logger.info(f"RMSE_DISTANCE_CHIS: {rmse_distance}")

        #coppia n_iter, distance
        couple = [(n, rmse_distance)]
        logger.info(f"COUPLE(N_ITER,DISTANCE RMSE):  {couple}")
        
        if(n == 1500):
            logger.info(f"TENSORFLOW_END: {end}")
            logger.info(f"TEMPO_ESECUZIONE TENSORFLOW(IN EPOCH): {total_tensorflow}")

        #incremento n_iter
        n += 100

    logger.removeHandler(handler)

In [12]:
list(product(["Setosa","Versicolor","Virginica"],[1,75,200],[0.1,0.25,0.5],[0.1,0.5,1],[1e-5], repeat=1))

[('Setosa', 1, 0.1, 0.1, 1e-05),
 ('Setosa', 1, 0.1, 0.5, 1e-05),
 ('Setosa', 1, 0.1, 1, 1e-05),
 ('Setosa', 1, 0.25, 0.1, 1e-05),
 ('Setosa', 1, 0.25, 0.5, 1e-05),
 ('Setosa', 1, 0.25, 1, 1e-05),
 ('Setosa', 1, 0.5, 0.1, 1e-05),
 ('Setosa', 1, 0.5, 0.5, 1e-05),
 ('Setosa', 1, 0.5, 1, 1e-05),
 ('Setosa', 75, 0.1, 0.1, 1e-05),
 ('Setosa', 75, 0.1, 0.5, 1e-05),
 ('Setosa', 75, 0.1, 1, 1e-05),
 ('Setosa', 75, 0.25, 0.1, 1e-05),
 ('Setosa', 75, 0.25, 0.5, 1e-05),
 ('Setosa', 75, 0.25, 1, 1e-05),
 ('Setosa', 75, 0.5, 0.1, 1e-05),
 ('Setosa', 75, 0.5, 0.5, 1e-05),
 ('Setosa', 75, 0.5, 1, 1e-05),
 ('Setosa', 200, 0.1, 0.1, 1e-05),
 ('Setosa', 200, 0.1, 0.5, 1e-05),
 ('Setosa', 200, 0.1, 1, 1e-05),
 ('Setosa', 200, 0.25, 0.1, 1e-05),
 ('Setosa', 200, 0.25, 0.5, 1e-05),
 ('Setosa', 200, 0.25, 1, 1e-05),
 ('Setosa', 200, 0.5, 0.1, 1e-05),
 ('Setosa', 200, 0.5, 0.5, 1e-05),
 ('Setosa', 200, 0.5, 1, 1e-05),
 ('Versicolor', 1, 0.1, 0.1, 1e-05),
 ('Versicolor', 1, 0.1, 0.5, 1e-05),
 ('Versicolor', 1

In [13]:
len(list(product(["Setosa","Versicolor","Virginica"],[1,75,200],[0.1,0.25,0.5],[0.1,0.5,1],[1e-5], repeat=1)))

81

In [14]:
for i in list(product(["Setosa","Versicolor","Virginica"],[1,75,200],[0.1,0.25,0.5],[0.1,0.5,1],[1e-5], repeat=1)):
    #if i not in [('Setosa', 0.05, 0.1, 0.1),('Versicolor', 0.05, 0.1, 0.1),('Virginica', 0.05, 0.1, 0.1)]:
    main(*i)

100%|██████████| 100/100 [00:17<00:00,  5.73it/s]


100%|██████████| 100/100 [00:17<00:00,  5.74it/s]


100%|██████████| 100/100 [00:17<00:00,  5.68it/s]


100%|██████████| 100/100 [00:17<00:00,  5.76it/s]
